Santiago Rodriguez Avila

Juan Pablo Reyes Fajardo

# <p style="text-align: center;">Tarea 3</p>
## <p style="text-align: center;">(Montecarlo, SARSA, Q-Learning)
</p>

Para esta tarea se quiere estudiar el comportamiento de los algoritmos de aprendizaje por refuerzo de: Montecarlo, SARSA y Q-learning. Se observarán características, comportamiento y composición de estos. Se hablará de políticas on y off (que el agente sigue la misma política a evaluar o una diferente), todo esto partiendo del escenario de la tarea anterior, el juego de escaleras y serpientes.

## Inicialización del MDP:

El MDP formuldado en el módulo de Python describe un  juego de escaleras y serpientes. Este permite:

* Dada una acción en un estado, retornar la recompensa y el estado resultante en esa acción. Considera el caso especial del estado terminal.


* Ejecutar una política arbitraria.

Importación de módulos:

In [239]:
from types import MethodType
from escalerasyserpientes import EscalerasSerpientes
import copy
import numpy as np
import random
import pandas as pd

Inicialización del MDP con:
* Casillas de victoria en (80,100)
* Casillas de pérdida en (23,37,45,67,89)
* Recompensa por paso de 0


In [240]:
es=EscalerasSerpientes([80,100],[23,37,45,67,89],0)

Fucnion para visualizar estados del MDP, la política y los valores de Q:

In [241]:
def imprimir_MDP():
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        display(pd.DataFrame({"Política: ": list(es.policy.values()), "Q: ":list(es.Q.values())},index=es.states).round(decimals=3))

Valores iniciales:

In [242]:
imprimir_MDP()

,Política:,Q:
1,Ad,"{'Ad': 0, 'At': 0}"
2,Ad,"{'Ad': 0, 'At': 0}"
3,Ad,"{'Ad': 0, 'At': 0}"
4,Ad,"{'Ad': 0, 'At': 0}"
5,Ad,"{'Ad': 0, 'At': 0}"
6,Ad,"{'Ad': 0, 'At': 0}"
7,Ad,"{'Ad': 0, 'At': 0}"
8,Ad,"{'Ad': 0, 'At': 0}"
9,Ad,"{'Ad': 0, 'At': 0}"
10,Ad,"{'Ad': 0, 'At': 0}"


### Pruebas:

La estructura de retorno del MDP es la siguiente: Casilla de llegada, valor de la recompensa obtenida en el paso,  acción tomada, movimiento realizados (valor del dado)y la variable que muestra si el estado al que se llegó es un estado terminal. En ese orden de ideas, las pruebas serían:

Probar función de pasos para movimientos determinísticos del agente para verificar victorias, perdidas, rebotes, escaleras y serpientes.

In [261]:
state_prueba=97
action_prueba=5
pruebas=[['Caso serpiente',96,2],
         ['Caso rebote inicio',2,-6],
         ['Caso rebote fin y serpiente',97,5],
         ['Caso victoria',94,6],
         ['Caso terminal',100,0],
         ['Caso fracaso',35,2],
         ['Caso terminal',37,0],
         ['Caso escalera',49,1]]
for prueba in pruebas:
    state_, reward, action, real_action, done=es.step(prueba[1], prueba[2], random=False)

    display(f'{prueba[0]}: Comienza en: {prueba[1]}, se mueve: {prueba[2]}, termina en {state_} y obtiene recompensa {reward}')

'Caso serpiente: Comienza en: 96, se mueve: 2, termina en 28 y obtiene recompensa 0'

'Caso rebote inicio: Comienza en: 2, se mueve: -6, termina en 6 y obtiene recompensa 0'

'Caso rebote fin y serpiente: Comienza en: 97, se mueve: 5, termina en 28 y obtiene recompensa 0'

'Caso victoria: Comienza en: 94, se mueve: 6, termina en 100 y obtiene recompensa 0'

'Caso terminal: Comienza en: 100, se mueve: 0, termina en Azul y obtiene recompensa 1.0'

'Caso fracaso: Comienza en: 35, se mueve: 2, termina en 37 y obtiene recompensa 0'

'Caso terminal: Comienza en: 37, se mueve: 0, termina en Rojo y obtiene recompensa -1.0'

'Caso escalera: Comienza en: 49, se mueve: 1, termina en 91 y obtiene recompensa 0'

Probar función de pasos para acciones del agente para verificar el movimiento aleatorio dado una acción "adelante" o "atrás":

In [278]:
state_prueba=98
accion_prueba='Ad'
for i in range(3**2):
    state_, reward, action, real_action, done=es.step(state_prueba, accion_prueba, random=True)

    display(f'Comienza en: {state_prueba}, acción: {accion_prueba}, termina en {state_} y obtiene recompensa {reward}')

'Comienza en: 98, acción: Ad, termina en 97 y obtiene recompensa 0'

'Comienza en: 98, acción: Ad, termina en 99 y obtiene recompensa 0'

'Comienza en: 98, acción: Ad, termina en 28 y obtiene recompensa 0'

'Comienza en: 98, acción: Ad, termina en 97 y obtiene recompensa 0'

'Comienza en: 98, acción: Ad, termina en 96 y obtiene recompensa 0'

'Comienza en: 98, acción: Ad, termina en 100 y obtiene recompensa 0'

'Comienza en: 98, acción: Ad, termina en 28 y obtiene recompensa 0'

'Comienza en: 98, acción: Ad, termina en 28 y obtiene recompensa 0'

'Comienza en: 98, acción: Ad, termina en 97 y obtiene recompensa 0'

## Generadores de episodios:

### Generador de episodios regular:

Juega el tablero usando la poítica del MDP y termina hasta que gana o pierde.

Retorna el número de pasos y el estado de la casilla final.

In [210]:
def episodio():
    state=1
    steps=0
    while True:
        state_, _,_, _, done=es.step(state, es.policy[state], random=True)
        state=state_
        steps+=1
        if done:
            return state_,steps


### Generador de episodios MonteCarlo

Juega el tablero usando la política en el argumento y termina hasta que gana o pierde.

Retorna listas con los estados, recompensas, acciones intentadas, movimientos, terminalidad, probabilidades de la política usada y número de pasos.

In [211]:
def episodio_MC(policy):
    state=1
    steps=0
    state_s=[1]
    rewards=[]
    tried_actions=[]
    real_actions=[]
    dones=[]
    probs=[]
    while True:
        pol,prob=policy(state)

        state_, reward,try_action, real_action, done=es.step(state, pol, random=True)
        state_s.append(state_)
        rewards.append(reward)
        real_actions.append(real_action)
        dones.append(done)
        probs.append(prob)
        tried_actions.append(try_action)
        
        state=state_
        
        steps+=1
        
        if done:
            return state_s,rewards,tried_actions,real_actions,dones,probs,steps

### Múltiples episodios:

In [212]:
def n_episodios(n):
    counter=0
    stepss=[]
    for i in range(n):
        state_,steps=episodio()
        if state_=='Azul':
            counter+=1
            stepss.append(steps)
    print(f'Victorias: {100*counter/n}% Pasos promedio: {sum(stepss)/len(stepss)}')

In [213]:
n_episodios(1000)

Victorias: 11.8% Pasos promedio: 18.93220338983051


## MonteCarlo Off-Policy

### Política de comportamiento (MonteCarlo):

Retorna la acción y la probabilidad de la acción.

Selecciona acción usando Q y epsilon greedy

In [214]:
def random_policy(state):
    return random.choice(list(es.Q[state].keys())),1/2

### Control con MonteCarlo Off-Policy para estimar $\pi^*:$

In [223]:
def MC_offpolicy(es,gamma):
    #Genera el episodio
    state_s,rewards,tried_actions,real_actions,dones,probs,steps=episodio_MC(random_policy)
    G=0
    W=1
    for i in reversed(range(len(state_s[0:-2]))):
        G=gamma*G+rewards[i+1]
        es.C[state_s[i]][tried_actions[i]]=es.C[state_s[i]][tried_actions[i]]+W
        es.Q[state_s[i]][tried_actions[i]]=es.Q[state_s[i]][tried_actions[i]]+(W/(es.C[state_s[i]][tried_actions[i]]))*(G-es.Q[state_s[i]][tried_actions[i]])
        es.policy[state_s[i]]=es.key_max(es.Q[state_s[i]])[0]

        if tried_actions[i]!=es.policy[state_s[i]]:
            break

        W=W*(1/probs[i])

### Inicialización y aprendizaje:

In [224]:
es.Q=es.init_Q(0,0)
es.C=es.init_Q(0,0)
es.policy = dict.fromkeys(es.states, 'Ad')
for i in range(0,100000):
    MC_offpolicy(es,0.9)

In [225]:
imprimir_MDP()

,Política:,Q:
1,Ad,"{'Ad': 0.32046860522342746, 'At': 0.2588466466..."
2,At,"{'Ad': 0.2020888108441245, 'At': 0.28874495135..."
3,At,"{'Ad': 0.20697500923054274, 'At': 0.3280491187..."
4,At,"{'Ad': 0.24472057856754245, 'At': 0.3730559567..."
5,At,"{'Ad': 0.2960421433078876, 'At': 0.38800220944..."
6,At,"{'Ad': 0.22904647773227957, 'At': 0.2731604618..."
7,Ad,"{'Ad': 0.285798268750259, 'At': 0.242121502029..."
8,Ad,"{'Ad': 0, 'At': 0}"
9,Ad,"{'Ad': 0.5411672315823821, 'At': 0.01859763118..."
10,Ad,"{'Ad': 0.42566671327809724, 'At': 0.2239722678..."


In [229]:
n_episodios(1000)

Victorias: 94.3% Pasos promedio: 76.40721102863202
